In [ ]:
! pip install torch torchvision transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import BertTokenizer
from transformers import BertModel
from PIL import Image
import requests
from io import BytesIO
import torch.optim as optim
import torch.nn as nn
import os
import pickle
from torchvision import models
from PIL import Image
import torch.nn.functional as F

In [ ]:
from torchvision import transforms
from transformers import BertTokenizer
from PIL import Image
import requests
from io import BytesIO

In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/ads_project/coco_predictions.csv')
"""
df = pd.read_csv('coco_predictions.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'coco_predictions.csv'

In [ ]:
class SimpleMultimodalModel(nn.Module):
    def __init__(self, num_classes, embedding_dim=128):
        super(SimpleMultimodalModel, self).__init__()

        # Text encoder: Embedding layer + GRU
        self.embedding = nn.Embedding(30522, embedding_dim)  # Using 30522 for BERT's Base uncased tokenizer
        self.gru = nn.GRU(embedding_dim, 256, batch_first=True)

        # Image encoder: Simplified CNN
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fusion layer and classifier
        self.fc1 = nn.Linear(64 + 256, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, input_ids, attention_mask, images):
        # Process text input through embedding and GRU
        x_text = self.embedding(input_ids)
        _, x_text = self.gru(x_text)
        x_text = x_text.squeeze(0)  # Remove sequence dimension

        # Process images through simplified CNN
        x_img = F.relu(self.conv1(images))
        x_img = F.relu(self.conv2(x_img))
        x_img = F.relu(self.conv3(x_img))
        x_img = self.adaptive_pool(x_img)
        x_img = torch.flatten(x_img, 1)  # Flatten all dimensions except batch

        # Fusion and classification
        combined_features = torch.cat((x_img, x_text), dim=1)
        fused_features = F.relu(self.fc1(combined_features))
        logits = self.fc2(fused_features)

        # Apply sigmoid to convert logits to probabilities
        probabilities = torch.sigmoid(logits)
        return probabilities



In [ ]:
class CocoCaptionsDataset(Dataset):
    def __init__(self, dataframe, transform=None, tokenizer=None):
        self.dataframe = dataframe
        self.transform = transform
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_url = row['coco_url']
        caption = eval(row['captions'])[0]  # Evaluating the string to get the list and taking the first item

        # Image processing
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')  # Convert image to RGB
        if self.transform:
            image = self.transform(image)

        # Text processing
        inputs = self.tokenizer(caption, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        input_ids, attention_mask = inputs['input_ids'].squeeze(0), inputs['attention_mask'].squeeze(0)

        # Labels - extracting the last 29 columns as classes
        labels = torch.tensor(row[2:].values.astype(float), dtype=torch.float32)

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'images': image}, labels


In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def preprocess_image(url):
    # Load image from URL
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = transform(img).unsqueeze(0)  # Add batch dimension
    return img

def prepare_text(text):
    # Tokenize text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs['input_ids'], inputs['attention_mask']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Initialize model, optimizer, and loss function
model = SimpleMultimodalModel(num_classes=29)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
optimizer = optim.Adam(model.parameters(), lr=0.1)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Process 3 examples from the DataFrame to see if the probabilies are correct
for index, row in df.head(3).iterrows():
    image_url = row['coco_url']
    caption = eval(row['captions'])[0]

    # Preprocess the image and text
    image_tensor = preprocess_image(image_url)
    input_ids, attention_mask = prepare_text(caption)

    # Forward pass through the model
    probabilities = model(input_ids, attention_mask, image_tensor)

    # Convert logits to probabilities and print (optional sigmoid and thresholding could be added)
    print(f"Example {index + 1}:")
    print(probabilities)


Example 1:
tensor([[0.5201, 0.4956, 0.5199, 0.4982, 0.5017, 0.4900, 0.4947, 0.4940, 0.5237,
         0.5034, 0.4908, 0.5001, 0.4932, 0.4887, 0.4988, 0.4944, 0.4721, 0.4883,
         0.4867, 0.4937, 0.5062, 0.5114, 0.5131, 0.5113, 0.5035, 0.5134, 0.4962,
         0.4847, 0.4969]], grad_fn=<SigmoidBackward0>)
Example 2:
tensor([[0.5122, 0.4986, 0.5192, 0.4994, 0.5076, 0.4894, 0.4922, 0.4911, 0.5207,
         0.5002, 0.4878, 0.5008, 0.4858, 0.4942, 0.5054, 0.4927, 0.4632, 0.4869,
         0.4814, 0.4924, 0.4994, 0.5074, 0.5191, 0.5087, 0.5044, 0.5177, 0.5034,
         0.4821, 0.4910]], grad_fn=<SigmoidBackward0>)
Example 3:
tensor([[0.5179, 0.5003, 0.5219, 0.5060, 0.5054, 0.4932, 0.4890, 0.4962, 0.5251,
         0.5001, 0.4857, 0.5017, 0.4950, 0.4931, 0.5057, 0.4995, 0.4728, 0.4902,
         0.4834, 0.5013, 0.4957, 0.5138, 0.5202, 0.5030, 0.5051, 0.5173, 0.5007,
         0.4744, 0.4898]], grad_fn=<SigmoidBackward0>)


In [ ]:
# Sample 10% of the data
sampled_df = df.sample(frac=0.1, random_state=42)  # Ensures reproducibility

# Proceed with dataset and dataloader setup
sampled_dataset = CocoCaptionsDataset(sampled_df, transform=transform, tokenizer=tokenizer)
sampled_dataloader = DataLoader(sampled_dataset, batch_size=64, shuffle=True)


In [ ]:
# test the data loader
for i, data in enumerate(sampled_dataloader):
    print(f"Batch {i+1}, Images shape: {data[0]['images'].shape}")
    if i == 1:  # Just check the first couple of batches
        break

Batch 1, Images shape: torch.Size([64, 3, 224, 224])
Batch 2, Images shape: torch.Size([64, 3, 224, 224])


In [ ]:
checkpoint_path = "student_checkpoint.pth.tar"

import pickle

log_file_path = './log.pickle'


In [ ]:
def save_checkpoint(model, optimizer, epoch, loss):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)

    with open(log_file_path, 'wb') as f:
        pickle.dump(log, f)
def load_checkpoint(model, optimizer):
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        loss = checkpoint['loss']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")

        with open(log_file_path, 'rb') as f:
            log = pickle.load(f)
        return start_epoch, loss, log
    else:
        print("No checkpoint found. Starting from epoch 1")
        return 0, None, {'epoch':[], 'loss':[]}

In [ ]:
start_epoch, previous_loss, log = load_checkpoint(model, optimizer)

No checkpoint found. Starting from epoch 1


In [ ]:
start_epoch, previous_loss, log

(0, None, {'epoch': [], 'loss': []})

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
import time
from tqdm import tqdm
loss_history = []  # List to store the average loss of each epoch
# Define the checkpoint interval
checkpoint_interval = 1800 #every 30 minutes one checkpoint, since 1 epoch takes around 1 hour
last_checkpoint_time = time.time()

start_epoch, previous_loss, log = load_checkpoint(model, optimizer)

num_epochs = 20

for epoch in range(start_epoch, num_epochs):
    running_loss = 0.0

    with tqdm(enumerate(sampled_dataloader), total=len(sampled_dataloader)) as pbar:
        for i, batch in pbar:
            inputs, labels = batch
            optimizer.zero_grad()
            logits = model(inputs['input_ids'], inputs['attention_mask'], inputs['images'])
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Update the progress bar description
            pbar.set_description(f"Epoch {epoch+1}, Loss: {running_loss / (i+1):.4f}")

            # Checkpoint based on time
            current_time = time.time()
            if current_time - last_checkpoint_time >= checkpoint_interval:
                save_checkpoint(model, optimizer, epoch, running_loss / (i + 1))
                last_checkpoint_time = current_time

    # Calculate average loss for the epoch
    average_loss = running_loss / len(sampled_dataloader)
    print(f"Epoch {epoch+1}, Average Loss: {average_loss:.4f}")
    loss_history.append(average_loss)  # Store the average loss for plotting
    # Update log and save checkpoint
    log['epoch'].append(epoch + 1)
    log['loss'].append(average_loss)
    save_checkpoint(model, optimizer, epoch, average_loss)

print("Training completed.")




In [ ]:
torch.save(model.state_dict(), "trained_student_model.pth")

In [ ]:
# Plotting the loss
plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Training Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()
